## Vegetable Price Prediction

Given data about **vegetables sold in a market**, let's try to predict the **price** of a given vegetable.

We will use a variety of regression models to make our predictions.

Data source: https://www.kaggle.com/datasets/sudipsamanta35/vegetable-market

### Getting Started

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import warnings
warnings.simplefilter('ignore')

In [6]:
data = pd.read_csv('Vegetable_market.csv')
data

,Vegetable,Season,Month,Temp,Deasaster Happen in last 3month,Vegetable condition,Price per kg
0,potato,winter,jan,15,no,fresh,20
1,tomato,winter,jan,15,no,fresh,50
2,peas,winter,jan,15,no,fresh,70
3,pumkin,winter,jan,15,no,fresh,25
4,cucumber,winter,jan,15,no,fresh,20
...,...,...,...,...,...,...,...
116,brinjal,winter,jan,15,yes,fresh,33
117,ginger,winter,jan,15,no,fresh,88
118,potato,summer,apr,32,no,fresh,24
119,peas,summer,apr,33,no,fresh,33


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Vegetable                        121 non-null    object
 1   Season                           121 non-null    object
 2   Month                            121 non-null    object
 3   Temp                             121 non-null    int64 
 4   Deasaster Happen in last 3month  121 non-null    object
 5   Vegetable condition              121 non-null    object
 6   Price per kg                     121 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 6.7+ KB


### Preprocessing

In [9]:
df = data.copy()
df

,Vegetable,Season,Month,Temp,Deasaster Happen in last 3month,Vegetable condition,Price per kg
0,potato,winter,jan,15,no,fresh,20
1,tomato,winter,jan,15,no,fresh,50
2,peas,winter,jan,15,no,fresh,70
3,pumkin,winter,jan,15,no,fresh,25
4,cucumber,winter,jan,15,no,fresh,20
...,...,...,...,...,...,...,...
116,brinjal,winter,jan,15,yes,fresh,33
117,ginger,winter,jan,15,no,fresh,88
118,potato,summer,apr,32,no,fresh,24
119,peas,summer,apr,33,no,fresh,33


In [12]:
# Binary encoding
df['Deasaster Happen in last 3month'] = df['Deasaster Happen in last 3month'].replace({'no':0, 'yes':1})

In [16]:
# Clean vegetable condition column
df['Vegetable condition'] = df['Vegetable condition'].replace({'scarp':'scrap'})

In [18]:
# Ordinal Encoding
df['Month'] = df['Month'].replace(
{
    'jan': 1,
    'apr': 4,
    'july': 7,
    'sept': 9,
    'oct': 10,
    'dec': 12,
    'may': 5,
    'aug': 8,
    'june': 6,
    ' ': np.NaN,
    'march': 3
}
)

In [19]:
df

,Vegetable,Season,Month,Temp,Deasaster Happen in last 3month,Vegetable condition,Price per kg
0,potato,winter,1.0,15,0,fresh,20
1,tomato,winter,1.0,15,0,fresh,50
2,peas,winter,1.0,15,0,fresh,70
3,pumkin,winter,1.0,15,0,fresh,25
4,cucumber,winter,1.0,15,0,fresh,20
...,...,...,...,...,...,...,...
116,brinjal,winter,1.0,15,1,fresh,33
117,ginger,winter,1.0,15,0,fresh,88
118,potato,summer,4.0,32,0,fresh,24
119,peas,summer,4.0,33,0,fresh,33


In [22]:
# Fill missing month values with column mode
df['Month'] = df['Month'].fillna(df['Month'].mode()[0])

In [24]:
df.isna().sum()

Vegetable                          0
Season                             0
Month                              0
Temp                               0
Deasaster Happen in last 3month    0
Vegetable condition                0
Price per kg                       0
dtype: int64

In [25]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix='Vegetable')
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [30]:
# One-hot encoding 
for column in ['Vegetable', 'Season', 'Vegetable condition']:
    df = onehot_encode(df, column)

In [33]:
# Split df into X and y
y = df['Price per kg']
X = df.drop('Price per kg', axis=1)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

# Scaling X
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

X_train

,Month,Temp,Deasaster Happen in last 3month,Vegetable_Bitter gourd,Vegetable_Raddish,Vegetable_brinjal,Vegetable_cabage,Vegetable_califlower,Vegetable_chilly,Vegetable_cucumber,...,Vegetable_radish,Vegetable_tomato,Vegetable_autumn,Vegetable_monsoon,Vegetable_spring,Vegetable_summer,Vegetable_winter,Vegetable_avarage,Vegetable_fresh,Vegetable_scrap
80,-0.807171,0.135584,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,...,-0.223607,-0.346410,-0.156174,-0.408248,-0.19245,-0.669534,1.024100,2.236068,-1.452966,-0.427900
38,1.067549,0.569452,1.628550,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,...,-0.223607,-0.346410,-0.156174,2.449490,-0.19245,-0.669534,-0.976467,-0.447214,0.688247,-0.427900
19,0.130189,0.894854,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,...,-0.223607,-0.346410,-0.156174,-0.408248,-0.19245,1.493576,-0.976467,-0.447214,0.688247,-0.427900
120,0.130189,0.786387,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,...,4.472136,-0.346410,-0.156174,-0.408248,-0.19245,1.493576,-0.976467,-0.447214,0.688247,-0.427900
27,0.130189,1.437189,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,...,-0.223607,-0.346410,-0.156174,-0.408248,-0.19245,1.493576,-0.976467,-0.447214,0.688247,-0.427900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,-0.807171,-1.057554,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,...,-0.223607,2.886751,-0.156174,-0.408248,-0.19245,-0.669534,1.024100,-0.447214,0.688247,-0.427900
72,2.629816,-1.057554,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,...,-0.223607,-0.346410,-0.156174,-0.408248,-0.19245,-0.669534,1.024100,-0.447214,-1.452966,2.336993
12,-0.807171,-1.057554,-0.614043,-0.251577,-0.251577,-0.156174,-0.301511,3.316625,-0.156174,-0.156174,...,-0.223607,-0.346410,-0.156174,-0.408248,-0.19245,-0.669534,1.024100,-0.447214,0.688247,-0.427900
107,2.629816,-0.406752,-0.614043,3.974921,-0.251577,-0.156174,-0.301511,-0.301511,-0.156174,-0.156174,...,-0.223607,-0.346410,-0.156174,-0.408248,-0.19245,-0.669534,1.024100,-0.447214,-1.452966,2.336993


In [34]:
X_train.mean()

Month                             -2.907727e-17
Temp                               1.982541e-17
Deasaster Happen in last 3month    1.057355e-17
Vegetable_Bitter gourd             8.855350e-17
Vegetable_Raddish                  7.401487e-17
Vegetable_brinjal                 -2.907727e-17
Vegetable_cabage                  -3.172066e-17
Vegetable_califlower              -1.321694e-18
Vegetable_chilly                  -5.022437e-17
Vegetable_cucumber                -3.965082e-17
Vegetable_garlic                   3.766828e-17
Vegetable_ginger                  -5.286776e-17
Vegetable_okra                    -3.965082e-17
Vegetable_onion                   -9.516197e-17
Vegetable_peas                    -4.229421e-17
Vegetable_pointed grourd           6.344132e-17
Vegetable_potato                  -6.344132e-17
Vegetable_pumkin                   9.251859e-18
Vegetable_radish                  -5.352861e-17
Vegetable_tomato                   7.533656e-17
Vegetable_autumn                  -6.079

In [35]:
X_train.var()

Month                              1.012048
Temp                               1.012048
Deasaster Happen in last 3month    1.012048
Vegetable_Bitter gourd             1.012048
Vegetable_Raddish                  1.012048
Vegetable_brinjal                  1.012048
Vegetable_cabage                   1.012048
Vegetable_califlower               1.012048
Vegetable_chilly                   1.012048
Vegetable_cucumber                 1.012048
Vegetable_garlic                   1.012048
Vegetable_ginger                   1.012048
Vegetable_okra                     1.012048
Vegetable_onion                    1.012048
Vegetable_peas                     1.012048
Vegetable_pointed grourd           1.012048
Vegetable_potato                   1.012048
Vegetable_pumkin                   1.012048
Vegetable_radish                   1.012048
Vegetable_tomato                   1.012048
Vegetable_autumn                   1.012048
Vegetable_monsoon                  1.012048
Vegetable_spring                

In [31]:
{column: list(df[column].unique()) for column in df.select_dtypes('object').columns}

{}

### Training

In [37]:
models = {
    "Linear Regression": LinearRegression(),
    "Linear Regression (L2 Regularization)": Ridge(),
    "Linear Regression (l1 Regularization)": Lasso(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "Support Vector Machine (RBF Kernel)": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBoost": XGBRegressor(),
    "LightGBM": LGBMRegressor(),
    "CatBoost": CatBoostRegressor(verbose=0)
}

In [38]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

Linear Regression trained.
Linear Regression (L2 Regularization) trained.
Linear Regression (l1 Regularization) trained.
K-Nearest Neighbors trained.
Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
Support Vector Machine (RBF Kernel) trained.
Decision Tree trained.
Random Forest trained.
Gradient Boosting trained.
XGBoost trained.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 30
[LightGBM] [Info] Number of data points in the train set: 84, number of used features: 6
[LightGBM] [Info] Start training from score 55.333333
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

### Results

In [39]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test)))

Linear Regression R^2 Score: 0.71258
Linear Regression (L2 Regularization) R^2 Score: 0.71197
Linear Regression (l1 Regularization) R^2 Score: 0.70461
K-Nearest Neighbors R^2 Score: 0.23805
Neural Network R^2 Score: -0.28811
Support Vector Machine (Linear Kernel) R^2 Score: 0.41883
Support Vector Machine (RBF Kernel) R^2 Score: -0.12694
Decision Tree R^2 Score: 0.59517
Random Forest R^2 Score: 0.66362
Gradient Boosting R^2 Score: 0.62869
XGBoost R^2 Score: 0.63068
LightGBM R^2 Score: 0.15834
CatBoost R^2 Score: 0.62010
